In [1]:
import pandas as pd
import numpy as np
import os
from scipy.sparse import csr_matrix
import scipy.sparse as sp
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import string
from string import digits
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


[nltk_data] Downloading package stopwords to /Users/arodi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/arodi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/arodi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#separate training data and labels
#print(os.path)
train_data_path='/Users/arodi/Desktop/Fall-2021/CMPE-255/project/CMPE-255_S3-Group10/C50/C50train/'
#change the data pathname while running based on location
author_list=os.listdir(train_data_path)#lists all the authors in path 'train_data_path'
train_labels=[]
author_text_tuple=[]
train_set=[]
authindex=0
#print('l',l)
for p in author_list:
    #print(p)
    if (not p.startswith('.')):
        file_list=os.listdir(train_data_path+p)#lists all the files in the path "train_data_path+p"
        #file_list.remove('.ipynb_checkpoints') # remove .ipynb_checkpoints if data is present  in jupyter notebook
        #file_list.remove('.DS_store') # remove .DS_store files if data is prsent in local drive
        path2_each_author=train_data_path+p#path for every author folder
        #print('file_list',file_list) #print all text files names for each author
        
        for author in author_list:
            if (not author.startswith('.')):  # this condition is to keep out all hidden files
                if author not in train_labels:
                    train_labels.append(author)
        writings = ''
        for file in range(len(file_list)):
            #print(path2_each_author+'/'+file)
            with open (path2_each_author+'/'+file_list[file],'r') as fp:
                writings+=fp.read()
                #author_text_tuple.append((author_list[authindex],file_list[file],(fp.read().splitlines())))
        train_set.append(writings)
        #authindex+=1
print("Number of train labels",len(train_labels))
print("Train labels",train_labels)#list of authors which are class labels
print("-"*50)
#print(author_text_tuple)# list of tuples with filename,content
#print(train_set)# print combined text from each author

        
#separate test data and labels
#print(os.path)
test_data_path='/Users/arodi/Desktop/Fall-2021/CMPE-255/project/CMPE-255_S3-Group10/C50/C50test/'
author_list=os.listdir(test_data_path)#lists all the authors in path 'train_data_path'
test_labels=[]
author_text_tuple=[]
test_set=[]
authindex=0
#print('l',l)
for p in author_list:
    #print(p)
    if (not p.startswith('.')):
        file_list=os.listdir(test_data_path+p)#lists all the files in the path "test_data_path+p"
        #file_list.remove('.ipynb_checkpoints') # remove .ipynb_checkpoints if data is present  in jupyter notebook
        #file_list.remove('.DS_store') # remove .DS_store files if data is prsent in local drive
        path2_each_author=test_data_path+p#path for every author folder
        #print('file_list',file_list)
        
        for author in author_list:
            if (not author.startswith('.')):  # this condition is to keep out all hidden files
                if author not in test_labels:
                    test_labels.append(author)
        writings = ''
        for file in range(len(file_list)):
            #print(path2_each_author+'/'+file)
            with open (path2_each_author+'/'+file_list[file],'r') as fp:
                writings+=fp.read()
                #author_text_tuple.append((author_list[authindex],file_list[file],(fp.read().splitlines())))
        test_set.append(writings)
        #authindex+=1
print("Number of test labels:",len(train_labels))
print("Test labels",train_labels)#list of authors which are class labels
print("-"*50)
#print(author_text_tuple)# list of tuples with filename,content
#print(test_set)# print combined text from each author 

# some data preprocessing todo



Number of train labels 50
Train labels ['RobinSidel', 'LynnleyBrowning', 'KouroshKarimkhany', 'MichaelConnor', 'JoeOrtiz', 'EricAuchard', 'AaronPressman', 'SimonCowell', "LynneO'Donnell", 'EdnaFernandes', 'KevinMorrison', 'SamuelPerry', 'PatriciaCommins', 'JohnMastrini', 'JanLopatka', 'KevinDrawbaugh', 'KarlPenhaul', 'MartinWolk', 'ScottHillis', 'DavidLawder', 'FumikoFujisaki', 'MarcelMichelson', 'NickLouth', 'DarrenSchuettler', 'WilliamKazer', 'TanEeLyn', 'PierreTran', 'HeatherScoffield', 'MureDickie', 'RogerFillion', 'JimGilchrist', 'BradDorfman', 'AlanCrosby', 'JonathanBirt', 'BenjaminKangLim', 'TheresePoletti', 'KeithWeir', 'JoWinterbottom', 'MarkBendeich', 'JaneMacartney', 'MatthewBunce', 'ToddNissen', 'PeterHumphrey', 'TimFarrand', 'SarahDavison', 'GrahamEarnshaw', 'BernardHickey', 'KirstinRidley', 'AlexanderSmith', 'LydiaZajc']
--------------------------------------------------
Number of test labels: 50
Test labels ['RobinSidel', 'LynnleyBrowning', 'KouroshKarimkhany', 'MichaelC

In [3]:
remover = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6})|\S*@\S*\s?;')


def preprocess(reviews):
    processed_text = []
    for review in reviews:
        # remove html tags  and email
        removed = re.sub(remover, '', review)
        to_remove = str.maketrans('', '', digits)
        number_removed = removed.translate(to_remove)
        #remove url
        url_removed = re.sub('r^https?:\/\/.*[\r\n]*', '', number_removed, flags=re.MULTILINE)
        #punctuation removed
        punctuation_removed=url_removed.translate(str.maketrans('', '', string.punctuation))
        # lowercase
        lower = punctuation_removed.lower()
        # lemmatize
        clean_text = ""
        stop_removed = ""
        lemmatized_text = ""
        # remove stop words using dictionary dictionary is faster than sets
        words_to_remove = stopwords.words('english')
        stopword_list = Counter(words_to_remove)
        tokens = word_tokenize(lower)
        tokens = [t.strip() for t in tokens]
        words_to_keep = [t for t in tokens if t not in stopword_list]
        stop_removed = " ".join(words_to_keep)
        # lemmatization is done based on context
        lemmatizer = WordNetLemmatizer()
        tokenization = nltk.word_tokenize(stop_removed)
        for w in tokenization:
            clean_text += (lemmatizer.lemmatize(w))+" "
        processed_text.append(clean_text.strip())
    
    return processed_text


In [4]:
# clean train and test data
processed_train = preprocess(train_set)
print(len(processed_train))
processed_test = preprocess(test_set)
print(len(processed_test))

50
50


In [5]:
# vectorize the writings
count_vectorizer = CountVectorizer(strip_accents='ascii',ngram_range=(1, 2))
cvmattrain = count_vectorizer.fit_transform(processed_train)
cvmattest = count_vectorizer.transform(processed_test)


#normalize data
tfidf_trasnformer =  TfidfTransformer()
tfidftrain = tfidf_trasnformer.fit_transform(cvmattrain)
tfidftest = tfidf_trasnformer.fit_transform(cvmattest)

#convert to csr matrix

compressed_train = csr_matrix(tfidftrain)
compressed_test = csr_matrix(tfidftest)

print(compressed_train.shape)
print(compressed_test.shape)

(50, 410102)
(50, 410102)


In [6]:
print("----------Linear SVC-----------")#0.88
#train the classifier, using linear svc
lsvcclf  = LinearSVC(C=10,max_iter=4000)
lsvcclf.fit(compressed_train, train_labels)
predicted_classes = lsvcclf.predict(compressed_test) # actual prediction of labels for test data
#print(predicted_classes)
accuracy_lsvc = accuracy_score(test_labels,predicted_classes)
f1_lsvc = f1_score(test_labels,predicted_classes, average='micro')
precision_lsvc = precision_score(test_labels,predicted_classes, average='micro')
recall_lsvc = recall_score(test_labels,predicted_classes, average='micro')
print("Accuracy:",accuracy_lsvc)
print("F1 score:",f1_lsvc)
print("Recall score:",recall_lsvc)
print("Precision score:",precision_lsvc)
# prepare a classification report
print(classification_report(test_labels, predicted_classes,zero_division=1))

----------Linear SVC-----------
Accuracy: 0.88
F1 score: 0.88
Recall score: 0.88
Precision score: 0.88
                   precision    recall  f1-score   support

    AaronPressman       1.00      1.00      1.00         1
       AlanCrosby       1.00      1.00      1.00         1
   AlexanderSmith       1.00      0.00      0.00         1
  BenjaminKangLim       1.00      1.00      1.00         1
    BernardHickey       1.00      1.00      1.00         1
      BradDorfman       1.00      1.00      1.00         1
 DarrenSchuettler       1.00      0.00      0.00         1
      DavidLawder       1.00      0.00      0.00         1
    EdnaFernandes       1.00      1.00      1.00         1
      EricAuchard       1.00      1.00      1.00         1
   FumikoFujisaki       1.00      1.00      1.00         1
   GrahamEarnshaw       1.00      1.00      1.00         1
 HeatherScoffield       0.50      1.00      0.67         1
       JanLopatka       1.00      0.00      0.00         1
    JaneMac

In [7]:
print("----------Naive Bayes-----------") #0.88
#train the classifier, using mnb
mnbclf  = MultinomialNB()
mnbclf.fit(compressed_train, train_labels)
predicted_classes_mnb = mnbclf.predict(compressed_test) # actual prediction of labels for test data
#print(predicted_classes)
accuracy_mnb = accuracy_score(test_labels,predicted_classes_mnb)
f1_mnb = f1_score(test_labels,predicted_classes_mnb, average='micro')
precision_mnb = precision_score(test_labels,predicted_classes_mnb, average='micro')
recall_mnb = recall_score(test_labels,predicted_classes_mnb, average='micro')
print("Accuracy:",accuracy_mnb)
print("F1 score:",f1_mnb)
print("Recall score:",recall_mnb)
print("Precision score:",precision_mnb)
# prepare a classification report
print(classification_report(test_labels, predicted_classes_mnb,zero_division=1))


----------Naive Bayes-----------
Accuracy: 0.88
F1 score: 0.88
Recall score: 0.88
Precision score: 0.88
                   precision    recall  f1-score   support

    AaronPressman       1.00      1.00      1.00         1
       AlanCrosby       1.00      1.00      1.00         1
   AlexanderSmith       1.00      0.00      0.00         1
  BenjaminKangLim       1.00      0.00      0.00         1
    BernardHickey       1.00      1.00      1.00         1
      BradDorfman       1.00      1.00      1.00         1
 DarrenSchuettler       1.00      0.00      0.00         1
      DavidLawder       1.00      0.00      0.00         1
    EdnaFernandes       1.00      1.00      1.00         1
      EricAuchard       1.00      1.00      1.00         1
   FumikoFujisaki       1.00      1.00      1.00         1
   GrahamEarnshaw       1.00      1.00      1.00         1
 HeatherScoffield       0.50      1.00      0.67         1
       JanLopatka       1.00      1.00      1.00         1
    JaneMa

In [8]:
print("----------Logistic Regression-----------")#0.86
#train the classifier, Logistic regression
lrclf  = LogisticRegression(C=10)
lrclf.fit(compressed_train, train_labels)
predicted_classes_lr = lrclf.predict(compressed_test) # actual prediction of labels for test data
#print(predicted_classes)
accuracy_lr = accuracy_score(test_labels,predicted_classes_lr)
f1_lr = f1_score(test_labels,predicted_classes_lr, average='micro')
precision_lr = precision_score(test_labels,predicted_classes_lr, average='micro')
recall_lr = recall_score(test_labels,predicted_classes_lr, average='micro')
print("Accuracy:",accuracy_lr)
print("F1 score:",f1_lr)
print("Recall score:",recall_lr)
print("Precision score:",precision_lr)
# prepare a classification report
print(classification_report(test_labels, predicted_classes_lr,zero_division=1))


----------Logistic Regression-----------
Accuracy: 0.86
F1 score: 0.8599999999999999
Recall score: 0.86
Precision score: 0.86
                   precision    recall  f1-score   support

    AaronPressman       1.00      1.00      1.00         1
       AlanCrosby       1.00      1.00      1.00         1
   AlexanderSmith       1.00      0.00      0.00         1
  BenjaminKangLim       1.00      0.00      0.00         1
    BernardHickey       1.00      1.00      1.00         1
      BradDorfman       1.00      1.00      1.00         1
 DarrenSchuettler       1.00      0.00      0.00         1
      DavidLawder       1.00      0.00      0.00         1
    EdnaFernandes       1.00      1.00      1.00         1
      EricAuchard       1.00      1.00      1.00         1
   FumikoFujisaki       1.00      1.00      1.00         1
   GrahamEarnshaw       1.00      1.00      1.00         1
 HeatherScoffield       0.50      1.00      0.67         1
       JanLopatka       1.00      0.00      0.0